<a href="https://colab.research.google.com/github/NUROISEA/anime-webui-colab/blob/main/waifu_diffusion_webui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Running the web ui

#@markdown # **⚠ Important ⚠**: Do not close this tab while running the web UI!

#@markdown Click the play button on the left to install the web UI and it's dependencies

#@markdown It takes around 3-5 minutes before you can see a `gradio` link to the web UI

#@markdown ![](https://i.imgur.com/Wg4K3rf.png)


!pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl

!git clone https://github.com/camenduru/stable-diffusion-webui
!git clone https://github.com/yfszzx/stable-diffusion-webui-images-browser /content/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
%cd /content/stable-diffusion-webui

!wget https://huggingface.co/hakurei/waifu-diffusion-v1-3/resolve/main/wd-v1-3-float32.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/wd-v1-3-float32.ckpt

!python launch.py --share --force-enable-xformers
#@markdown ----

In [ ]:
#@title Saving your generated images

#@markdown Check this before running this cell if you want to save it your Google Drive
SAVE_TO_GOOGLE_DRIVE = False #@param { 'type': 'boolean' }

#@markdown The folder path to save the zip file, if the above is checked, if left blank it will be in `AI/Generated`
GOOGLE_DRIVE_FOLDER_PATH = "AI/Generated" #@param { 'type': 'string' }

#@markdown If you didnt choose to save to your Google Drive, you can find `output.zip` at the files tab of this collab

#@markdown ![](https://i.imgur.com/fJpZpsS.png)
%cd /content/stable-diffusion-webui

output_folder = "/content"
message = "You can now open the files tab and download "
message_end = ".zip!"

if SAVE_TO_GOOGLE_DRIVE:
  from google.colab import drive
  drive.mount('/content/drive')

  if GOOGLE_DRIVE_FOLDER_PATH == "":
    output_folder = "/content/drive/MyDrive/AI/Generated"
  else:
    output_folder = "/content/drive/MyDrive/" + GOOGLE_DRIVE_FOLDER_PATH

  message = "You can now open " + GOOGLE_DRIVE_FOLDER_PATH + " in Google Drive and find "
  message_end = ".zip to download!"

  %mkdir -p {output_folder}

from datetime import datetime
str_date = datetime.today().strftime('%Y-%m-%d-%H%M%S')
archive_name = "outputs-" + str_date

!zip -r {output_folder}/{archive_name} outputs

print("********************************************************")
print("********************************************************")
print("Files saved!\n" + message + archive_name + message_end)
print("********************************************************")
print("********************************************************")